# Imports

In [3]:
#misc
import logging
import os
import pprint
logging.basicConfig(level=logging.INFO)

import multiprocessing
print("cores", multiprocessing.cpu_count())
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# http
import requests
import json
from datetime import datetime, date, timedelta
import time
from bs4 import BeautifulSoup


# data viz
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

# data
import numpy as np
#import dask.dataframe as pd
import pandas as pd
import pickle
from collections import defaultdict
import re
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', -1)
from copy import deepcopy

#nlp
import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS
from sense2vec import Sense2VecComponent
from spacy import displacy
from tqdm import tqdm_notebook as tqdm


import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.ldamodel import LdaModel
from gensim.models.phrases import Phrases, Phraser

import pyLDAvis.gensim
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

# Constants
RANDOM_STATE=42
SAMPLE_SIZE=2000
WORKERS=3

cores 8


INFO:gensim.summarization.textcleaner:'pattern' package not found; tag filters are not available for English


# Original data

In [13]:
df_full = pd.read_json('/Users/kirill.kogan/Documents/data/reddit/nosleep.txt',lines=True)

# Clean

## clean it up

In [14]:
def clean_frame(frame):
    df = frame.copy()
    
    ##############
    # Duplicates
    ##############
    df = df.drop_duplicates(subset='id')
    
    #######
    # NAs
    #######
    df = df.dropna(subset=['selftext'])
    #df = df[df['selftext'] != '[removed]']
    #df = df[df['selftext'] != '[deleted]']
    df = df[df['selftext'].str.len() > 300]
    df = df[df['score'] > 0]
    
    ########
    # Text cleaning
    ########
    df['selftext'] = df['selftext'].replace(value = '', regex = 'amp;\\S+') 
    df = df[df['title'].str.contains('voting thread') != True]
    
    ##################
    # Date processing
    ##################
    df['created_utc'] = pd.to_datetime(df['created_utc'], unit='s')
    df['created_year'] = pd.DatetimeIndex(df['created_utc']).year
    df['created_month'] = pd.DatetimeIndex(df['created_utc']).month
    #df['edited'] = pd.to_datetime(df['edited'], unit='s')
    #df['retrieved_on'] = pd.to_datetime(df['retrieved_on'], unit='s')
    #df['author_created_utc'] = pd.to_datetime(df['author_created_utc'], unit='s')

    mask = ['author',
        'created_utc',
        'created_year',
        'created_month', 
        'score', 
        'num_comments',
        'full_link', 
        'title', 
        'selftext',]
    
    return df[mask] 

In [15]:
df_clean = clean_frame(df_full)


In [16]:
df_clean.sort_values(['score'], ascending=False, inplace=True)

In [17]:
df_clean.to_pickle("/Users/kirill.kogan/Documents/data/reddit/reddit_df_clean.pkl")

## Load clean pickle

In [4]:
df_clean = pd.read_pickle('/Users/kirill.kogan/Documents/data/reddit/reddit_df_clean.pkl')

In [13]:
df_19_sub = df_clean[df_clean['created_year'] == 2019].head(200)
df_19_sub.to_pickle('./reddit_df_19_sub.pkl')

In [18]:
df_clean_sub = df_clean.head(25000).sample(3200, random_state=RANDOM_STATE)
df_clean_sub.to_pickle('./reddit_df_clean_sub.pkl')

## Check terms

In [5]:
df_clean[df_clean['selftext'].str.contains('ragna')]

author         created_utc  created_year  created_month  score  \
257018  wysteria1 2019-06-03 08:06:59  2019          6              62      
212316  duofinu17 2018-06-16 00:18:52  2018          6              20      
267357  dnzprs    2019-08-19 10:15:48  2019          8              10      

        num_comments  \
257018  14             
212316  6              
267357  8              

                                                                                                  full_link  \
257018  https://www.reddit.com/r/nosleep/comments/bw86hu/the_end_of_times/                                    
212316  https://www.reddit.com/r/nosleep/comments/8rfnic/flyingeddys_blog/                                    
267357  https://www.reddit.com/r/nosleep/comments/csf4at/im_being_watched_by_fivedimensional_beings_part_i/   

                                                        title  \
257018  The End of Times                                        
212316  FlyingEddy's Blog.                                      
267357  I'm being watched by five-dimensional beings - Part I   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   